In [1]:
#Import the geemap and google earth engine module 
import geemap
import ee

In [19]:
Map = geemap.Map(center=[38.8,23.5], zoom=10)
Map

Map(center=[38.8, 23.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [22]:
#Select your Study Area by drwaing a bounding box from the map

feature = Map.draw_last_feature
aoi = feature.geometry()

# Define time interval for the Sentinel - 1 data

#Before Fire (Start Date)
before_start='2021-07-20'
before_end='2021-07-30'

#After Fire (End Date)
after_start='2021-08-25'
after_end='2021-09-05'

#Parameters for SAR

polarization ="VH";            #Preferred polarization for Fire mapping, but you can also select 'VV'

pass_direction="ASCENDING";   #Or DESCENDING consider changing this parameter if your image collection is empty

difference_threshold=1.15;     #Threshold to be applied on the difference image, leave default

#Dont make changes in the below section

#Load and Select Sentinel-1 data by predefined level
# Load and filter Sentinel-1 GRD data by predefined parameters
collection= ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.eq('instrumentMode','IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
  .filter(ee.Filter.eq('orbitProperties_pass',pass_direction)) \
  .filter(ee.Filter.eq('resolution_meters',10)) \
  .filterBounds(aoi) \
  .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start,after_end)

# Create a mosaic of selected tiles and clip to study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply reduce the radar speckle by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

#------------------------------- FIRE EXTENT MAPPING -------------------------------#

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the fire extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# I discovered that some crop fields produce misleading pixels.
# Include ESA Landcover layer to mask cropland, water and built-up areas
sfire = ee.ImageCollection("ESA/WorldCover/v100").first()
sfire_mask = sfire.gte(40).updateMask(sfire.gte(40))

#Burned layer where crop land, water and builtup area is assigned a 0 value
burned_mask = difference_binary.where(sfire_mask,0)
# Burned layer without pixels in perennial crop lands, water and builtup area
burned = burned_mask.updateMask(burned_mask)

# Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
# This operation reduces noise of the fire extent product
connections = burned.connectedPixelCount()
burned = burned.updateMask(connections.gte(34))

Map.addLayer(before_filtered, {'min':-25, 'max':0}, 'Before Fire',0)
Map.addLayer(after_filtered, {'min':-25, 'max':0}, 'After Fire',1)
Map.addLayer(difference,{'min':0, 'max':2},"Difference Layer",0)
Map.addLayer(burned,{'palette':'ffce00'},'Burned Area')

In [18]:
import os

In [31]:
# Set output directory
out_dir = os.path.expanduser('~/Downloads')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

filename = os.path.join(out_dir, 'burned.tif')

geemap.ee_export_image(before_filtered, filename=filename, scale=35, region=aoi, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Abu Bakar\Downloads
